# Compute data summary statistics
Now that we've organized and cleaned the question data, let's compute some statistics.

In [7]:
# load data
import pandas as pd
# data = pd.read_csv('../../data/reddit_data/combined_data_question_data.gz', sep='\t', compression='gzip')
comment_data = pd.read_csv('../../data/reddit_data/advice_subreddit_filter_comment_question_data.gz', sep='\t', compression='gzip')
post_data = pd.read_csv('../../data/reddit_data/subreddit_submissions_2018-01_2019-12.gz', sep='\t', compression='gzip', index_col=False, usecols=['id', 'selftext'])
post_data.rename(columns={'selftext' : 'article_text', 'id' : 'parent_id'}, inplace=True)
data = pd.merge(comment_data, post_data, on=['parent_id'], how='left')
print(f'{data.shape[0]} total questions')
print(f'{data.loc[:, "parent_id"].nunique()} total posts')

734967 total questions
272467 total posts


## Text statistics

In [8]:
# load tokenizer
# import torch
# model_tokenizer = torch.load('../../data/reddit_data/BART_tokenizer.pt')
from nltk.tokenize import WordPunctTokenizer
word_tokenizer = WordPunctTokenizer()
data = data.assign(**{
    'post_tokens' : data.loc[:, 'article_text'].apply(lambda x: word_tokenizer.tokenize(x)),
    'question_tokens' : data.loc[:, 'question'].apply(lambda x: word_tokenizer.tokenize(x)),
})

In [9]:
import numpy as np
post_len = data.loc[:, "post_tokens"].apply(lambda x: len(x)).values
question_len = data.loc[:, "question_tokens"].apply(lambda x: len(x)).values
print(f'mean post length = {np.mean(post_len)} +/- {np.std(post_len)}')
print(f'mean question length = {np.mean(question_len)} +/- {np.std(question_len)}')

mean post length = 304.5176422887014 +/- 220.76208617123916
mean question length = 13.887307865523214 +/- 8.07912311345707


## Author data
What % of the data includes metadata about authors?

In [69]:
author_data = pd.read_csv('../../data/reddit_data/author_data/combined_author_prior_comment_data.gz', sep='\t', compression='gzip')
# fix date var
from datetime import datetime
author_data = author_data.assign(**{'date_day_bin' : pd.Series(author_data.loc[:, 'date_day_bin'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d')).values, dtype='object')})
# author_data = author_data.assign(**{'date_day_bin' : author_data.loc[:, 'date_day_bin'].apply(lambda x: x.to_pydatetime()).values})
display(author_data.head())

/home/ianbstew/miniconda3/envs/py3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1,2,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,author,date_day,subreddit,expert_pct,relative_time,expert_pct_bin,relative_time_bin,age,location,location_region,date_day_bin,text_embed,subreddit_embed
0,Battleboy43,2018-06-02,pcmasterrace,0.0,78340.0,0.0,0.0,-1.0,UNK,UNK,2018-01-01 00:00:00,NaN,NaN
1,m4xk0,2018-07-13,pcmasterrace,0.0,69552.0,0.0,0.0,-1.0,UNK,UNK,2018-07-01 00:00:00,NaN,NaN
2,hAbadabadoo22,2019-02-06,AmItheAsshole,0.0,8878.0,0.0,0.0,-1.0,UNK,UNK,2019-01-01 00:00:00,NaN,NaN
3,haysu-christo,2018-11-04,personalfinance,0.0,86011.0,0.0,1.0,-1.0,UNK,UNK,2018-07-01 00:00:00,NaN,NaN
4,haysu-christo,2018-11-05,personalfinance,0.0,74117.0,0.0,0.0,-1.0,UNK,UNK,2018-07-01 00:00:00,NaN,NaN


In [11]:
valid_authors = set(author_data.loc[:, 'author'].unique())
valid_author_data = data[data.loc[:, 'author'].isin(valid_authors)]
print(f'{valid_author_data.shape[0]/data.shape[0]*100}% questions can be connected to an author with some kind of data')

64.78739861789713% questions can be connected to an author with some kind of data


Any difference for discrete and continuous representation?

In [12]:
continuous_vars = ['text_embed', 'subreddit_embed']
valid_continuous_var_authors = set(author_data.dropna(subset=continuous_vars, how='all').loc[:, 'author'].unique())
continuous_var_author_data = data[data.loc[:, 'author'].isin(valid_continuous_var_authors)]
print(f'{continuous_var_author_data.shape[0]/data.shape[0]*100}% questions can be connected to an author with a continuous representation')

8.82883177067814% questions can be connected to an author with a continuous representation


## Text+author data
What is the representation of authors in the text data?

We want % of comments with each reader group and embedding type.

In [42]:
import pandas as pd
from datetime import datetime
import pytz
post_author_data = pd.read_csv('../../data/reddit_data/combined_data_question_data.gz', 
                               sep='\t', index_col=False, compression='gzip',
                               usecols=['article_id', 'created_utc', 'id', 'author'])
post_author_data = post_author_data.assign(**{'date' : post_author_data.loc[:, 'created_utc'].apply(lambda x: datetime.fromtimestamp(x, tz=pytz.utc).replace(tzinfo=None))})
## convert to date day bins
from importlib import reload
import data_helpers
reload(data_helpers)
from data_helpers import assign_date_bin
import numpy as np
author_date_bins = author_data.loc[:, 'date_day_bin'].unique()
author_date_bins = np.array(list(map(lambda x: x.timestamp(), author_date_bins)))
post_author_data = post_author_data.assign(**{
    'date_day_bin' : post_author_data.loc[:, 'date'].apply(lambda x: assign_date_bin(x.timestamp(), author_date_bins, convert_timezone=False))
})
display(post_author_data.head())

,article_id,created_utc,id,author,date,date_day_bin
0,e7jfrg,1575750384,fa1p58y,Cthulhu1960,2019-12-07 20:26:24,2019-07-01 00:00:00
1,dvg6no,1573592095,f7d39h9,kingerthezinger,2019-11-12 20:54:55,2019-07-01 00:00:00
2,ds8qsk,1573002293,f6oimpf,choose_a_username-2,2019-11-06 01:04:53,2019-07-01 00:00:00
3,du3qhn,1573342846,f71pzmt,gracehv2018,2019-11-09 23:40:46,2019-07-01 00:00:00
4,ax01sr,1551655611,ehqcd5n,coniferous-1,2019-03-03 23:26:51,2019-01-01 00:00:00


In [71]:
dynamic_author_vars = ['relative_time_bin', 'expert_pct_bin', 'text_embed', 'subreddit_embed']
static_author_vars = ['location_region']
combined_author_post_data = post_author_data.copy()
for dynamic_author_var_i in dynamic_author_vars:
    combined_author_post_data = pd.merge(combined_author_post_data, author_data.loc[:, [dynamic_author_var_i, 'author', 'date_day_bin']], on=['author', 'date_day_bin'], how='left')
for static_var_i in static_author_vars:
    combined_author_post_data = pd.merge(combined_author_post_data, author_data.loc[:, [static_author_var_i, 'author']], on='author', how='left')

MemoryError: Unable to allocate 267. GiB for an array with shape (35830712595,) and data type int64

### Debug: joining with embedding data

Why are we losing so many of the embeddings?

In [11]:
## where are all the embeddings going?
from datetime import datetime
tmp_author_data = author_data.drop(['subreddit_embed', 'text_embed'], axis=1)
tmp_author_data = tmp_author_data.assign(**{
    'date_day' : tmp_author_data.loc[:, 'date_day'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
})
from ast import literal_eval
from importlib import reload
import data_helpers
reload(data_helpers)
from data_helpers import assign_date_bin
author_embeddings_data_file = '../../data/reddit_data/author_data/author_date_embeddings_type=subreddit.gz'
author_embeddings_data = pd.read_csv(author_embeddings_data_file, sep='\t', compression='gzip', index_col=False)
embed_var = list(filter(lambda x: x.endswith('_embed'), author_embeddings_data.columns))[0]
author_embeddings_data = author_embeddings_data.assign(**{embed_var : author_embeddings_data.loc[:, embed_var].apply(lambda x: literal_eval(x))})
author_embeddings_data = author_embeddings_data.assign(**{'date_day_bin' : author_embeddings_data.loc[:, 'date_day_bin'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))})
## join w/ author data via date
embedding_date_bins = author_embeddings_data.loc[:, 'date_day_bin'].apply(lambda x: x.timestamp()).unique()
tmp_author_data = tmp_author_data.assign(**{
    'date_day_bin' : tmp_author_data.loc[:, 'date_day'].apply(lambda x: assign_date_bin(x.timestamp(), embedding_date_bins))
})
tmp_author_data = pd.merge(tmp_author_data, author_embeddings_data.loc[:, ['author', 'date_day_bin', embed_var]], on=['author', 'date_day_bin'], how='left')

In [24]:
print(f'{tmp_author_data.dropna(subset=["subreddit_embed"], axis=0).loc[:, "author"].nunique()}/{author_embeddings_data.loc[:, "author"].nunique()} authors retained')

22148/55609 authors retained


In [72]:
# display(tmp_author_data.loc[:, 'date_day_bin'].head())
# display(author_embeddings_data.loc[:, 'date_day_bin'].head())
x = tmp_author_data.loc[:, 'date_day_bin'].iloc[0]
y = author_embeddings_data.loc[:, 'date_day_bin'].iloc[0]
print(x)
print(type(x))
print(y)
print(type(y))

2018-01-01 00:00:00+00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2018-01-01 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [68]:
from importlib import reload
import data_helpers
reload(data_helpers)
from data_helpers import assign_date_bin
embedding_date_bin = author_embeddings_data.loc[:, 'date_day_bin'].apply(lambda x: x.timestamp()).unique()
test_date = datetime.strptime('2018-03-01', '%Y-%m-%d')
test_date = test_date.timestamp()
# print(dir(test_date))
test_bin_date = assign_date_bin(test_date, embedding_date_bin)
print(embedding_date_bins)
print(test_bin_date)

[1.5147648e+09 1.5304032e+09 1.5463008e+09 1.5619392e+09]
2018-01-01 00:00:00+00:00


In [78]:
datetime.strptime(test_bin_date.strftime('%Y-%m-%d'), '%Y-%m-%d')

datetime.datetime(2018, 1, 1, 0, 0)

In [80]:
from pytz import utc
test_date_bin = author_embeddings_data.loc[:, 'date_day_bin'].iloc[0]
print(test_date_bin)
print(datetime.fromtimestamp(test_date_bin.timestamp(), tz=utc).replace(tzinfo=None))

2018-01-01 00:00:00
2018-01-01 00:00:00


In [58]:
# from datetime import timezone
from datetime import tzinfo
from datetime import timedelta
# timezone(datetime.timedelta(seceonds=0))
datetime.fromtimestamp(test_date_bin.timestamp())

TypeError: timestamp() takes no keyword arguments

In [60]:
help(datetime.fromtimestamp)

Help on built-in function fromtimestamp:

fromtimestamp(...) method of builtins.type instance
    timestamp[, tz] -> tz's local time from POSIX timestamp.

